# Example of the usage of the Bike Simulation Framework

This file will provide an example on how to use this simulation Framework and all the classes that were implemented for this.
***

| Class | Description  |
|------|------|
|   **BikeStationCDF** | This class will provide all comulative distribution functions for every Bike Stations on the Network.<br> Every station will have at least 7 CDF's which will correspond to every day of the week.|
|**BikeStations**|This class will provide the basic methods and attributes that describe <br> the actions and parameters from which a station can be defined.|
|**BikeRelocationScheme**|This class contains provides a calculation for the relocation weights <br> which correspond to how many bikes will be populated into a station when it reaches a critical level.|
|**RouteAnalysis**|This is a class that interacts with SUMO Simulator to retrive all <br> the objects from the SUMO network as well as all the required metadata|
|**Traci**|Python implementation of SUMO Simulator API's|
|**Geometry**|Distance Calculation, Angle Calculation and other Geometry APIs are provided my this class.|

In [ ]:
from BikeStationCDF import BikeStationCDF as BSCDF
from BikeStations import BikeStationNetwork as BSN
from BikeRelocationScheme import BikeRelocationScheme as BRS

from RouteAnalysis import RouteAnalysis

import traci
from sumolib import checkBinary

from Geometry import GeometryClass

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import math

## SUMO Server Connection

Simulation needs to be running at the time this python script is run. In the case of the current example, the server port is always set 58890 and the connection order is marked as 2 (SUMO requires that each client has its own different connection id).

In [ ]:
from sumolib import checkBinary


In [ ]:
# sumoBinary = checkBinary('sumo')
sumoBinary = checkBinary('sumo-gui')

# first, generate the route file for this simulation
# generate_routefile()

# This start a multi client connection, port and client number
# must be configured

traci.start([
    sumoBinary, '-c', 'main.sumocfg',
    '--tripinfo-output', 'output/tripinfo.xml',
    '--num-clients', '1',
    '--log', 'output/consoleOutput.txt',
    '--verbose', 'true',
    '--print-options', 'true',
    '--gui-testing-debug', 'true',
    '--ignore-route-errors', 'true'])

In [ ]:
C_NUMBEROFDOCKS = 20

from BikeStations import BikeStationNetwork as BSN
myBikeNetwork = BSN(C_NUMBEROFDOCKS)

In [ ]:
RouteAnalysisSUMO = RouteAnalysis()

In [ ]:
AllStationsIds = myBikeNetwork.getAllStationOnNetwork()
RelocationSchemes = BRS(AllStationsIds).getRebalancingWeights()

### BikeInitialStatus

This attribute will hold a list with the number of bikes that are available when simulation starts.

In [ ]:
myBikeNetwork.resetNetwork()
BikeInitialStatus = [s.availableBikes() for s in list(myBikeNetwork.BikeStationsDict.values())]
# BikeInitialStatus

### Method: updateStatusOfStations
- **Input: Time -> (float):**
    - The current time of the simulation which goes from 0.0 to 24.0
<br>
- **Input: dayItinerary -> (list(list)):**
    - A list of all the trips that will occur within a single day for every station

#### Details:
This method will take all the trips that are planned within the provided time window and the corresponding **BikeStation Class** APIs will be called to update the state of such Bike Stations.

#### Note:

* This class requires the push/remove bike APIs to be populated with the intended relocation weight in case it is needed, this information comes from BRS().getRebalancingWeights() API *

In [ ]:
IgnoreRoutes = [
    [50, 57],
    [58, 37],
    [50, 72],
    [31, 35],
    [54, 77],
    [52, 35],
    [50, 80],
    [80, 37],
    [37, 60],
    [6, 35],
    [51, 77]
]

ForceIgnore = [50, 37, 80, 35, 77]

In [ ]:
def sumoStartBikeTrip(bikeObject, time, destinationId):
    
    
    stationId = bikeObject.stationId
    edgeStart = myBikeNetwork.StationsOnNetwork[int(stationId)]
    
    if [int(stationId), int(destinationId)] not in IgnoreRoutes:
        if((int(stationId) not in ForceIgnore) and ((int(destinationId) not in ForceIgnore))):
            try:
                edgeEnd = myBikeNetwork.StationsOnNetwork[int(destinationId)]
            except:
                edgeEnd = 'startEdge'

            routeName = \
                'myBikeRoute_' + '_' +\
                str(sumoStartBikeTrip.bikeTripsID)

            vehicleName = \
                'BikeTrip_' + '_Station' +\
                str(stationId) + '_' + str(destinationId) + '_' + str(sumoStartBikeTrip.bikeTripsID)

            traci.route.add(routeName, [edgeStart, edgeEnd])

            traci.vehicle.add(vehicleName,
                              routeName,
                              typeID='bike')

            traci.vehicle.setColor(vehicleName,
                                   (0, 255, 0))
            traci.vehicle.setRoutingMode(
                vehicleName,
                traci.constants.ROUTING_MODE_AGGREGATED)

            traci.vehicle.changeTarget(
                vehicleName, edgeEnd)

            traci.vehicle.setParameter(vehicleName,
                                       'has.rerouting.device',
                                       'true')

            traci.vehicle.setParameter(vehicleName,
                                       'device.rerouting.period',
                                       '30')

            sumoStartBikeTrip.bikeTripsID += 1

    
sumoStartBikeTrip.bikeTripsID = 0

In [ ]:
def updateStatusOfStations(time, dayItinerary, arrivals2Ignore,weekDay, RelocationWeights):
    for idx in range(len(dayItinerary)):
        for eventsStation in range(len(dayItinerary[idx])):
            # StationId retrived from trip timestamps of dayItinerary
            StationId = int(dayItinerary[idx][eventsStation][2])
            # Process events only when they're equal to time
            # times are represented in seconds, they need to be multiplied
            # by C_MINUTES_IN_HOUR
            if(int(
                    dayItinerary[idx][eventsStation][0] *
                    C_MINUTES_IN_HOUR *
                    C_SECONDS_IN_MINUTE *
                    SUMO_STEP_SIZE) == time):
                # print(dayItinerary[idx][eventsStation][0], time)

                # departure or arrival trip type
                tripInstance = dayItinerary[idx][eventsStation]
                # print(tripInstance) 
                if(tripInstance[1] == BSN.C_ARRIVALS_IDX):
                    if(arrivals2Ignore.get(str(StationId),0) == 0):
#                         print(RelocationWeights[
#                             StationId][weekDay][2])
                        if(myBikeNetwork.getBikeStationObject(
                                StationId).pushBike(
                                    RelocationWeights[
                                        StationId][weekDay][2]) is False):
                            # This should never be the case as the
                            # relocation occurs within the scope of
                            # push/removeBike()
                            print(myBikeNetwork.getBikeStationObject(                            
                                StationId).availableBikes())
                            raise
                    else:
                        # print(tripInstance)
                        arrivals2Ignore[str(StationId)] = \
                            int(arrivals2Ignore[str(StationId)]) - 1
                        if(int(arrivals2Ignore[str(StationId)]) < 0):
                            arrivals2Ignore[str(StationId)] = 0

                else:
#                     print(RelocationWeights[
#                         StationId][weekDay][2])
                    if(myBikeNetwork.getBikeStationObject(
                            StationId).removeBike(
                                    RelocationWeights[
                                        StationId][weekDay][2]) is False):
                        # This should never be the case as the
                        # relocation occurs within the scope of
                        # push/removeBike()
                        print(myBikeNetwork.getBikeStationObject(
                            StationId).availableBikes())
                        raise
                    else:
                        sumoStartBikeTrip(myBikeNetwork.getBikeStationObject(
                            StationId), time, int(tripInstance[3]))
    

### Method: getNetworkRiskStatus
- **Input: bikeNetwork -> (BikeStationNetwork Object):**
    - This is an object from **BikeStationNetwork Class** which will be used to iterate over all the bike stations status to get the number of available bikes.
<br>
<br>
- **Output: DockRiskStations, BikeRiskStations**
    - The output will be two lists that will contain the number of stations which number of docks is critical and another list that contains the stations which number of bikes is critical.

#### Details:
The output will be two lists that will contain the number of stations which number of docks is critical and another list that contains the stations which number of bikes is critical.

#### Note:

None

In [ ]:
def getNetworkRiskStatus(bikeNetwork, optimalValues, currentDay):
    # StatusBike Network -> [StationId, NumberofAvailableBikes]
    StatusBikeNetwork = [[
            int(s), 
            int(bikeNetwork.getBikeStationObject(s).availableBikes()),
            optimalValues[int(s)][currentDay][2] * 0.01 * C_NUMBEROFDOCKS]
            for s in bikeNetwork.BikeStationsDict.keys() ]
    
    data = np.array(StatusBikeNetwork)
    
    # Identify Risk Stations by applying a formula np.max/min +- 1 whcih has calculated empirically
    # to provide a mechanism for Risk Assesment. It's performance is out of the scope of this
    # module for now.
    
    #DockRiskStations -> Stations that have more bikes than docks
    DockRiskStations = data[(data[:,1] >= (np.max(data[:,1]) - 1)) & ((data[:,1]) > (data[:,2]))]
    
    #BikeRiskStations -> Stations that have more docks than bikes
    BikeRiskStations = data[(data[:,1] <= (np.min(data[:,1]) + 1)) & ((data[:,1]) < (data[:,2]))]

    return DockRiskStations, BikeRiskStations

### Method: upcomingTrips
- **Input: Time -> (float):**
    - The current time of the simulation which goes from 0.0 to 24.0
<br>
- **Input: dayItinerary -> (list(list)):**
    - A list of all the trips that will occur within a single day for every station
<br>
- **Output: dayUpcoming**
    - Returns a list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.

#### Details:
Returns a list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.

#### Note:

None

In [ ]:
def upcomingTrips(time, dayItinerary):
    dayUpcoming = []
    for s in range(len(dayItinerary)):
        UpcomingTrips_bool = \
            ((dayItinerary[s][:,0] * C_MINUTES_IN_HOUR *
              C_SECONDS_IN_MINUTE * SUMO_STEP_SIZE ) >= time) & \
            ((dayItinerary[s][:,0] * C_MINUTES_IN_HOUR *
              C_SECONDS_IN_MINUTE * SUMO_STEP_SIZE ) <
                 (time + (0.125 * C_MINUTES_IN_HOUR * C_SECONDS_IN_MINUTE * SUMO_STEP_SIZE)))
        
        # dayUpcoming -> [[list(Trips timestamps)], [dayItinerary bool indexers for location of those trips],
        # dayItineraryIdx]
        if(dayItinerary[s][UpcomingTrips_bool].size > 0):
            dayUpcoming.append([ dayItinerary[s][UpcomingTrips_bool],  UpcomingTrips_bool, s])
            
    return dayUpcoming

### Method: getDistanceBetweenStations
- **Input: RouteAnalysisSUMO -> (RouteAnalysis Object):**
    - Object from RouteAnalysis class that will be used to retrieve the coordinates of the edges to where the bike stations are and which edge centers are required.
<br>
- **Input: stationIdEdge1 -> (String):**
    - Name of the first station.
<br>
- **Input: stationIdEdge1 -> (String):**
    - Name of the seconds station.
<br>
- **Output: Distance between two edges**
    - Returns a **float** value with the distance between the two edges centers. The distance is in meters.

#### Details:
Returns a **float** value with the distance between the two edges centers. The distance is in meters.

#### Note:

None

In [ ]:
def getDistanceBetweenStations(RouteAnalysisSUMO, stationIdEdge1, stationIdEdge2):
    p1 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge1]]
    p2 = [float(s) for s in RouteAnalysisSUMO.EdgetoEdgeCenter[stationIdEdge2]]
    
    return GeometryClass.getDistance(p1,p2)

### Method: retrieveRiskMeasurements
- **Input: DockRiskStations**
    - A list that will contain the number of stations which number of docks is critical.
- **Input: UpcomingInfo**
    - A list of the trips that will be analyzed within the current time window. The current time window is equal to (0.125) * 60 minutes.
<br>
- **Input: bikeNetwork -> (BikeStationNetwork Object):**
    - This is an object from **BikeStationNetwork Class** which will be used to iterate over all the bike stations status to get the number of available bikes.
<br>
- **Input: RouteAnalysisSUMO -> (RouteAnalysis Object):**
    - Object from RouteAnalysis class that will be used to retrieve the coordinates of the edges to where the bike stations are and which edge centers are required.
<br>
- **Output: NearStations\[Idx\], UpcomingIdx**
    - Returns all the stations that are critical and which current scope trips could be swapped. A distance of no more than 400 meters between stations is consider to suggest these new destination stations. UpcomingIdx corresponds to a boolean list that maps the elements of UpcomingInfo input to the ones that where finally selected.

#### Details:
Returns all the stations that are critical and which current scope trips could be swapped. A distance of no more than 400 meters between stations is consider to suggest these new destination stations. UpcomingIdx corresponds to a boolean list that maps the elements of UpcomingInfo input to the ones that where finally selected.

#### Note:

None

In [ ]:
def retrieveRiskMeasurements(BikeRiskStation, UpcomingInfo, myBikeNetwork, RouteAnalysisSUMO):
    NearStations = []
    UpcomingIdx = []
    for i, StationInfo in enumerate(UpcomingInfo):
        for j, trip in enumerate(StationInfo[0]):
            
            # Only departures will be analyzed by this function
            if((int(trip[1]) != BSN.C_ARRIVALS_IDX) & (int(trip[3]) in myBikeNetwork.getAllStationOnNetwork())):
                distance = \
                    getDistanceBetweenStations(
                        RouteAnalysisSUMO, 
                        myBikeNetwork.StationsOnNetwork.get(int(trip[3])), 
                        myBikeNetwork.StationsOnNetwork[BikeRiskStation])
                
                # 400 meters were choosen as per Singla study where this is the
                # maximum distance users where willing to change the
                # destination station
                if((distance < 400.0) & (distance > 5.0)):
                    NearStations.append(str(BikeRiskStation))
                    UpcomingIdx.append([i, j])
                    # print(str(BikeRiskStation) + ' replaces: ' + str(int(trip[3])))
                    # print('One near station is: ' + str(int(trip[3])) + ' instead of: ' + str(RiskStation))
                    
    if(NearStations):
        # From all posible solutions, only the station with less bikes will be
        # selected as it is intended to avoid those stations to be empty
        RewardDestinationBikes =\
            [myBikeNetwork.getBikeStationObject(s).availableBikes() for s in NearStations]
        Idx = np.where(RewardDestinationBikes == np.min(RewardDestinationBikes))[0][0]
        
        return NearStations[Idx], UpcomingIdx
    
    else:
        return 0, 0

## Main Function


#### Details:

All functions previosly defined are used in order to find the critical dock stations, the ones that can be replaced are analyzed and then modified from the dayItinerary provided for the current time window.

#### Note:

None

### Relocation Analysis for Ideal participation

Two different participation percentages will be analyzed, 0% and 100%

### Comparison between rewards and non rewards model

In [ ]:
C_SIMULATION_DAYS = 1
C_REWARDMODEL_ON = True
C_DAYITINERARY = 0

SUMO_STEP_SIZE = int(1 / traci.simulation.getDeltaT())

SimulationResults = []
for simDay in range(C_SIMULATION_DAYS):

    dayItineraryObject = myBikeNetwork.getDayItinerary(C_DAYITINERARY)
    dayItinerary = []
    for s in dayItineraryObject:
        dayItinerary.append(s.astype(object))
     
    for testType in [C_REWARDMODEL_ON]:
        
        C_SECONDS_IN_MINUTE = 60
        C_MINUTES_IN_HOUR = 60
        C_HOURS_IN_DAY = 24
        C_OPTIMAL_LEVEL = C_NUMBEROFDOCKS * 0.50 # 10 bikes / 20 docks
        arrival2Ignore = {}


        # for idx, _ in enumerate(dayItinerary):
        #     print('Itinerary for Station: ' + str(dayItinerary[idx][0][2]))



        myBikeNetwork.resetNetwork()
        for time in range(
                C_SECONDS_IN_MINUTE *
                C_MINUTES_IN_HOUR *
                6 *
                SUMO_STEP_SIZE,
                C_SECONDS_IN_MINUTE *
                C_MINUTES_IN_HOUR *
                C_HOURS_IN_DAY *
                SUMO_STEP_SIZE):
            
            
            # SUMO step simulation
            traci.simulationStep()
            
            # Only run validation every 60 Seconds
            if((time % (C_SECONDS_IN_MINUTE * SUMO_STEP_SIZE)) == 0):

                # Get Most Critical Stations
                DockRiskStations, BikeRiskStations =\
                    getNetworkRiskStatus(myBikeNetwork, RelocationSchemes, C_DAYITINERARY)
                # Get the itinerary for the next time delta
                UpcomingInfo = upcomingTrips(time, dayItinerary)

                if(time % (int(
                        C_MINUTES_IN_HOUR *
                        C_SECONDS_IN_MINUTE * 
                        SUMO_STEP_SIZE * 
                        0.125)) == 0):
                    NearStation = 0
                    UpcomingIdx = [0, 0]
                    # print('\n\nIt ends ****************')
                    # print('********************** It begins \n\n')
                    for stationId in BikeRiskStations[:,0]:
                        if(C_REWARDMODEL_ON is testType):
                            
                            NearStation, UpcomingIdx =\
                                retrieveRiskMeasurements(
                                    int(stationId),
                                    UpcomingInfo,
                                    myBikeNetwork,
                                    RouteAnalysisSUMO)
                            if(NearStation != 0):
                                NearStationObject = myBikeNetwork.getBikeStationObject(
                                    int(NearStation))

                                curOptimalLevel =\
                                    (RelocationSchemes[int(NearStation)][C_DAYITINERARY][2]
                                     * 0.01 * C_NUMBEROFDOCKS)

                                ProblemAssesment = \
                                    int(abs( curOptimalLevel - NearStationObject.availableBikes()))
                                # print(ProblemAssesment)

                                if((ProblemAssesment >= len(UpcomingIdx))):
                                    UpcomingIdx = UpcomingIdx
                                elif(len(UpcomingIdx) > 0):
                                    UpcomingIdx = UpcomingIdx[0:ProblemAssesment]

                                if((len(UpcomingInfo) > 0)):
                                    for i, s in enumerate(UpcomingIdx):
                                        # Index for the three required elements of UpcomingInfo
                                        # Trip info, dayItineraryTripIndex and dayItineraryIdx
                                        idx_1 = s[0]
                                        # Index for the trips that are relevant within the UpcomingInfo
                                        # element
                                        idx_2 = s[1]
                                        dayItineraryIdx = UpcomingInfo[idx_1][2]
                                        dayItineraryStationIdx = UpcomingInfo[idx_1][1]
                                        UpcomingTrips = UpcomingInfo[idx_1][0]

                                        # print('Test #1: ' + str(UpcomingInfo[idx_1][0]))
                                        # print('Test #2: ' + str(dayItinerary[dayItineraryIdx]))

                                        for i, boolIdx in enumerate(dayItineraryStationIdx):
                                            if(
                                                (boolIdx == True) & \
                                                ((dayItinerary[dayItineraryIdx][i][1]) != BSN.C_ARRIVALS_IDX) &\
                                                (UpcomingInfo[idx_1][0][idx_2][3] == dayItinerary[dayItineraryIdx][i][3])):

                                                try:
                                                    arrival2Ignore[
                                                        str(int(dayItinerary[dayItineraryIdx][i][3]))] = \
                                                    int(arrival2Ignore[
                                                        str(int(dayItinerary[dayItineraryIdx][i][3]))]) + 1
                                                except:
                                                    arrival2Ignore.setdefault(
                                                        str(int(dayItinerary[dayItineraryIdx][i][3])), 1)

#                                                 print(
#                                                     'Time %.2f' % (time/60) + ': ' +
#                                                     'Station ' + str(int(dayItinerary[dayItineraryIdx][i][3])) +\
#                                                     ' is replaced by station: ' + str(NearStation))
                                                dayItinerary[dayItineraryIdx][i][3] = float(NearStation)
                                                if(myBikeNetwork.getBikeStationObject(
                                                        NearStation).pushBike(
                                                            RelocationSchemes[
                                                                int(NearStation)][C_DAYITINERARY][2]) is False):
                                                    # This should never be the case as the
                                                    # relocation occurs within the scope of
                                                    # push/removeBike()
                                                    print(myBikeNetwork.getBikeStationObject(                            
                                                        StationId).availableBikes())
                                                    raise

                                        

                                    # print('Test #3: ' + str(dayItinerary[dayItineraryIdx]))


                # Update the Stations based on the dayItinerary provided / modified
                updateStatusOfStations(
                    time,
                    dayItinerary, 
                    arrival2Ignore,
                    C_DAYITINERARY,
                    RelocationSchemes)

        #     if(time % C_MINUTES_IN_HOUR == 0):
        #         print('At time: ' + str(time) + ' minutes.') 
        #         print('Dock Risk Stations: ')
        #         print(DockRiskStations)
        #         print('Bike Risk Stations: ')
        #         print(BikeRiskStations)
        #         print()


        #Relocation events on every station
        RelocationFinalCounter =\
            [(s, myBikeNetwork.getBikeStationObject(s).getStationStats()[0])
             for s in myBikeNetwork.BikeStationsDict.keys() ]
        print('\n\nReward Model is ' + str(testType))
        print(RelocationFinalCounter)


        relocationStats = []
        for eachElement in RelocationFinalCounter:
            relocationStats.append(eachElement[1])
        # relocationStats    
        plt.hist(relocationStats)

        print(np.mean(relocationStats))
        print(np.array(relocationStats).sum())
        print(np.max(relocationStats))
        
        
        SimulationResults.append([
            np.mean(relocationStats), 
            np.array(relocationStats).sum(), 
            np.max(relocationStats)])

In [ ]:
traci.close()

In [ ]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 

# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("images/24hoursSUMO_Div1s.csv") 

# Preview the first 5 lines of the loaded data 
data.head()

In [ ]:
SUMOSimulatorOutput = list(data['# running vehicles [#]'])

In [ ]:
SizeAverage = 1500
myZeros = list(np.zeros(SizeAverage))
myZeros.extend(SUMOSimulatorOutput)
SUMOSimulatorOutput = myZeros

In [ ]:

N = SizeAverage
cumsum, maxValues = [0], []

for i, x in enumerate(SUMOSimulatorOutput, 1):
    cumsum.append(cumsum[i-1] + x)
    if i>=N:
        #moving_ave = (cumsum[i] - cumsum[i-N])/N
        max_value = max(SUMOSimulatorOutput[i-N:i])
        #can do stuff with moving_ave here
        #moving_aves.append(moving_ave)
        maxValues.append(max_value)
        
# plt.plot(maxValues)
# plt.plot(SUMOSimulatorOutput)

In [ ]:
N = int(SizeAverage*0.5)
cumsum, moving_aves = [0], []

for i, x in enumerate(maxValues, 1):
    cumsum.append(cumsum[i-1] + x)
    if i>=N:
        moving_ave = (cumsum[i] - cumsum[i-N])/N
        #can do stuff with moving_ave here
        moving_aves.append(moving_ave)


plt.plot(np.arange(0,24,24/len(moving_aves)),moving_aves)
plt.xlabel('Time of the day (h)')
plt.ylabel('Number of trips during the day')
plt.title('Trips along a simulation day on SUMO')
plt.figure(2)
plt.plot(SUMOSimulatorOutput)